# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [5]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
# confirm that our new database was created
db_list = mongo.list_database_names()

In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
collections = db.list_collection_names()
pprint(collections)

[]


In [10]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in the 'uk_food' database:")
for collection in collections:
    print("-", collection)

Collections in the 'uk_food' database:


In [11]:
# review a document in the establishments collection
document = db['establishments'].find_one()
pprint(document) 

None


In [12]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "name": "Penang Flavours",
    "location": "Greenwich",
    "type": "Halal",
    "rating": None  #new resturant
}

In [14]:
# Insert the new restaurant into the collection
result = establishments.insert_one(penang_flavours)

In [15]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print("Successfully added 'Penang Flavours' to the database.")
else:
    print("Failed to add the restaurant.")

Successfully added 'Penang Flavours' to the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [42]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find(
    {"BusinessType": "Restaurant/Cafe/Canteen"}, 
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  
)

In [41]:
# Print the result
for document in result:
    pprint(document)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [43]:
# Update the new restaurant with the correct BusinessTypeID

business_type_document = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "_id": 0})


business_type_id = None
if business_type_document:
    business_type_id = business_type_document['BusinessTypeID']


if business_type_id:
    update_result = establishments.update_one(
        {"name": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id}}
    )

In [44]:
close_matches = establishments.find(
    {"BusinessType": {"$regex": ".*Restaurant.*", "$options": "i"}},
    {"BusinessType": 1, "_id": 0}
)

for match in close_matches:
    pprint(match)

In [45]:
# Confirm that the new restaurant was updated
if business_type_document and 'BusinessTypeID' in business_type_document:
    if update_result.modified_count > 0:
        print("Successfully updated 'Penang Flavours' with the correct BusinessTypeID.")
    else:
        print("Failed to update 'Penang Flavours' or it already had the correct BusinessTypeID.")
else:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.")

BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [46]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthority": "Dover"})
print(f"Number of establishments in Dover: {dover_count}")


Number of establishments in Dover: 0


In [47]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthority": "Dover"})
print(f"Number of establishments deleted from Dover: {delete_result.deleted_count}")


Number of establishments deleted from Dover: 0


In [48]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthority": "Dover"})
print(f"Number of establishments in Dover after deletion: {dover_count_after}")


Number of establishments in Dover after deletion: 0


In [49]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()

if sample_document:
    pprint(sample_document)
else:
    print("No documents left in the 'establishments' collection.")

{'_id': ObjectId('64d284a173026ef6a6206baa'),
 'location': 'Greenwich',
 'name': 'Penang Flavours',
 'rating': None,
 'type': 'Halal'}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


In [50]:
longitude_update_result = establishments.update_many(
    {"longitude": {"$type": "string"}}, 
    [{"$set": {"longitude": {"$toDecimal": "$longitude"}}}]
)

print(f"Updated {longitude_update_result.modified_count} documents where 'longitude' was stored as a string.")


Updated 0 documents where 'longitude' was stored as a string.


In [51]:
latitude_update_result = establishments.update_many(
    {"latitude": {"$type": "string"}},
    [{"$set": {"latitude": {"$toDecimal": "$latitude"}}}]
)

print(f"Updated {latitude_update_result.modified_count} documents where 'latitude' was stored as a string.")


Updated 0 documents where 'latitude' was stored as a string.


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [52]:
# Change the data type from String to Integer for RatingValue
rating_update_result = establishments.update_many(
    {"RatingValue": {"$type": "string"}}, 
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

print(f"Updated {rating_update_result.modified_count} documents where 'RatingValue' was stored as a string.")


Updated 0 documents where 'RatingValue' was stored as a string.


In [53]:
# Check that the coordinates and rating value are now numbers
suspect_rating_document = establishments.find_one({"RatingValue": {"$type": "string"}})

if suspect_rating_document:
    pprint(suspect_rating_document)
else:
    print("No documents found where 'RatingValue' is stored as a string after the update.")


No documents found where 'RatingValue' is stored as a string after the update.
